# Nanopore Sequenz Analyse

> Sollten Sie diese Anleitung auf **GitHub** lesen, dann werden Sie merken das einige Code-Zeilen am Ende unvollstänig sind. Dennoch können Sie die Zellen vollständig in das Clipboard kopieren, wenn Sie sie durch Doppelklicken markieren.

Vorbereitende Installationen mit APT:

- `sudo apt install -y hdfview hdf5-tools ont-fast5-api dotter porechop minimap2 samtools igv`
- `wget https://cdn.oxfordnanoportal.com/software/analysis/ont_guppy_cpu_6.3.7-1~focal_amd64.deb`
- `sudo apt install -f ./ont_guppy_cpu_6.3.7-1~focal_amd64.deb`



In [1]:
pwd

/home/ubuntu/CompBiol3/NanoporeSeq


### Download Data Files

In [2]:
wget https://www.dropbox.com/s/si08bjwdvcmtuls/ALT327_c9ffe0f3_0.fast5

--2024-01-10 21:44:34--  https://www.dropbox.com/s/si08bjwdvcmtuls/ALT327_c9ffe0f3_0.fast5
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/si08bjwdvcmtuls/ALT327_c9ffe0f3_0.fast5 [following]
--2024-01-10 21:44:34--  https://www.dropbox.com/s/raw/si08bjwdvcmtuls/ALT327_c9ffe0f3_0.fast5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6fbdbce8cf3b05235b11238aea.dl.dropboxusercontent.com/cd/0/inline/CLEP6wWv5g0XAp1wlv7UbtPs7J-YbOndV4LGywMnAfcVlaNui3VLmsm0fFJFKaS_X2y0Xs7ukHXb4C1w7i8yMVvx1yXTHt2GSvrok9t5IhGWEnTs-PbYJEd0Dc-jDnjXhjg/file# [following]
--2024-01-10 21:44:35--  https://uc6fbdbce8cf3b05235b11238aea.dl.dropboxusercontent.com/cd/0/inline/CLEP6wWv5g0XAp1wlv7UbtPs7J-YbOndV4LGywMnAfcVlaNui3VLmsm0fFJFKaS_X2y0Xs7ukHXb4C1w7i8yMVv

In [3]:
wget https://www.dropbox.com/s/88hqohg3271un21/pBR322.fasta

--2024-01-10 21:44:58--  https://www.dropbox.com/s/88hqohg3271un21/pBR322.fasta
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/88hqohg3271un21/pBR322.fasta [following]
--2024-01-10 21:44:59--  https://www.dropbox.com/s/raw/88hqohg3271un21/pBR322.fasta
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc59eeadc32b5933b39c6c272b5c.dl.dropboxusercontent.com/cd/0/inline/CLGykFOYu6svixcSZR5NeM0XMT5JrPcXU2vUJrehF-xOYtwhVh2TefY7rq-dVK4YKUaXxtpWO8Qs45zrvW8cuaBZP9N_tjix9gs0BlNDJNW9vDUKANsk3rA4G9End9gD1D0/file# [following]
--2024-01-10 21:44:59--  https://uc59eeadc32b5933b39c6c272b5c.dl.dropboxusercontent.com/cd/0/inline/CLGykFOYu6svixcSZR5NeM0XMT5JrPcXU2vUJrehF-xOYtwhVh2TefY7rq-dVK4YKUaXxtpWO8Qs45zrvW8cuaBZP9N_tjix9gs0BlNDJNW9vDUKANsk3rA4

In [4]:
ls

ALT327_c9ffe0f3_0.fast5  Untitled.ipynb  pBR322.fasta


### Move Files to new Folder

In [5]:
mkdir OriginalFast5

In [6]:
cp ALT327_c9ffe0f3_0.fast5 OriginalFast5

### Betrachten der FAST5-Dateien
FAST5-Dateien sind Hierarchical Data Format 5 (HDF5)-Dateien mit einem spezifischen Aufbau, der von Oxford Nanopore Technologies (ONT) für die Speicherung von Sequenzierrohdaten definiert wurde und die von bspw. vom MinION erzeugt werden. Es gibt zwei FAST5-Typen: Single-FAST5 und Multi-FAST5. Eine Multi-FAST5-Datei enthält Daten zu mehreren Reads in einer einzigen Datei, während eine Single-FAST5-Datei Daten eines einzigen Reads enthält. Single-FAST5-Dateien werden kaum verwendet.
Ein freies Programm zur Betrachtung der Dateien ist HDFView, dass Sie für alle Betriebssysteme bei[https://hdfgroup.or] downloaden können. Wir haben es oben bereits mit APT installiert

Sie können die graphische Version mit `hdfview` im Terminal starten.

Wir nutzen hier aber Command Line Tools.

Dazu steht uns das Programm `h5ls` zur Verfügung , dass den Inhalt einer FAST5-Datein auflistet. Da die Datei des MinION eine Multi-FAST5-Datei ist, begrenzen wir die Ausgabe mit dem Befehl `head -3` auf drei Zeilen.
.

In [7]:
h5ls OriginalFast5/ALT327_c9ffe0f3_0.fast5 | head -3

read_0048d23d-4b49-476b-b18c-d102e7a3e393 Group
read_0077101c-2127-4fd4-96cc-9e9393bbb260 Group
read_009ca1ca-5af2-47ec-a267-ee88d9d7ff17 Group


Um zu prüfen, was für einen einzelnen Read gespeichert ist, können Sie diesen Read mit `h5ls` wie ein Verzeichnis, also getrennt mit einem Slash, angeben:


In [8]:
h5ls OriginalFast5/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393

Raw                      Group
channel_id               Group
context_tags             Group
tracking_id              Group


In [9]:
h5ls OriginalFast5/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393/Raw/Signal

Signal                   Dataset {6991/Inf}


Mit der Option `--data` können wir uns einzelne Datensätze anzeigen lassen. In einem "Unterverzeichnis" sind die Rohdaten des MinIONs enthalten, also beispielsweise die Spannungsänderungen über einer Pore (Squiggle-Plot):


In [10]:
h5ls -d OriginalFast5/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393/Raw/Signal | head -3

Signal                   Dataset {6991/Inf}
    Data:
                 Unable to print data.


**Upps**: *"Unable to print data."* Da ist etwas schief gelaufen!?! Oxford Nanopore verwendet zwei unterschiedliche Verfahren, um die Rohdaten in den FAST5-Dateien zu komprimieren: vbz oder gzip. Neuerdings wird eine proprietäre VBZ-Kompression von ONT verwendet, womit wiederum ältere Analyseprogramme manchmal Probleme haben. Mit der Programmsuite `ont-fast5-api` läßt sich die Kompression umwandeln und damit das Problem nicht lesbarer Daten (siehe oben) lösen. Die Installation haben wir bereits oben abgeschlossen.
N
Anschließend erfolgt die Umwandlung der Datenkompression, wobe*i fas*t5 das Verzeichnis mit den FAST5-Dateien bezeichnet und wir mit decompressed das Ausgabeverzeichnis benennen. Dann klappt die Datenextraktion:

In [11]:
compress_fast5 --input_path OriginalFast5 --save_path OriginalFast5Gzip --compression gzip

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
| 1 of 1|####################################################|100% Time: 0:00:13


In [12]:
h5ls -d OriginalFast5Gzip/ALT327_c9ffe0f3_0.fast5/read_0048d23d-4b49-476b-b18c-d102e7a3e393/Raw/Signal | head -3

Signal                   Dataset {6991}
    Data:
         727, 523, 529, 536, 542, 523, 524, 460, 460, 482, 471, 454, 481, 462,


**Tipp**: Um die Datenanalyse zu beschleunigen können die tausend Reads in der FAST5-Datei in einzelne FAST5-Dateien aufgetrennt werden. Auch das ist mit der Programmsuite `ont-fast5-api` möglich:

In [13]:
multi_to_single_fast5 --input OriginalFast5Gzip --save OriginalFast5GzipSingle

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
| 1 of 1|####################################################|100% Time: 0:00:03


Dabei bezeichnet *OriginalFast5Gzip* das Verzeichnis mit den FAST5-Dateien und *OriginalFast5GzipSingle* das Ausgabeverzeichnis. Anschließend können FAST5-Dateien mit beispielsweise je hundert Reads erstellt werden:


In [14]:
single_to_multi_fast5 --input OriginalFast5GzipSingle/0 --save OriginalFast5GzipMulti100 --batch_size 100

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
| 10 of 10|##################################################|100% Time: 0:00:07


Dabei bezeichnet die Option `--save` das Ausgabeverzeichnis und die Option `--batch_size` die Anzahl der Reads pro Datei, die jeweils den Namen *batch_0.fast5*, *batch_1.fast5*, *batch_2.fast5* usw. erhalten.


### Basecalling mit Guppy
Guppy ist ein Programm, das die Basecalling-Algorithmen von Oxford Nanopore Technologies und mehrere bioinformatische Anwendungen für die Nachbearbeitung enthält. Dazu zählen auch das Demultiplexing, Adapter-Trimming und Alignments mit Zielsequenzen, wie dem Plasmid pBR322. Es ist für Windows-, MacOS- und Linux-Betriebssysteme von Oxford Nanopore erhältlich. Die Installation ist weiter oben beschrieben. Neben dem Pfad zu einem Verzeichnis mit unseren FAST5-Dateie(n) (Option `--input_path`) benötigt der Basecaller den Namen eines Ausgabeverzeichnisses (`--save_path`) und die verwendete Flußzellen- (`--flowcell`) und Sequenzierkit- (`--kit`) Kombination, in unserem Fall also der Flongle (FLO-FLG001) und das Rapid-Barcoding Kit (SQK-RBK004). Das Ausgabeverzeichnis wird automatisch erstellt, wenn es nicht bereits existiert.

Auf meiner virtuellen Linux-Maschine in VitualBox auf einem MacBook Pro dauert das Basecallen von tausend Sequenzen rund eine Stunde mit einer CPU (3.478.643 ms entsprechen rund 58 min).


In [15]:
guppy_basecaller --input_path OriginalFast5 --flowcell FLO-FLG001 --kit SQK-RBK004 --save_path BasecalledAll

CRASHPAD MESSAGE: 
ONT Guppy basecalling software version 6.3.7+532d626, minimap2 version 2.22-r1101
config file:        /opt/ont/guppy/data/dna_r9.4.1_450bps_hac.cfg
model file:         /opt/ont/guppy/data/template_r9.4.1_450bps_hac.jsn
input path:         OriginalFast5
save path:          BasecalledAll
chunk size:         2000
chunks per runner:  256
minimum qscore:     9
records per file:   4000
num basecallers:    1
cpu mode:           ON
threads per caller: 4

Use of this software is permitted solely under the terms of the end user license agreement (EULA).By running, copying or accessing this software, you are demonstrating your acceptance of the EULA.
The EULA may be found in /opt/ont/guppy/bin
Found 1 input read file to process.
Init time: 1574 ms

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
Caller time: 3478643 ms, Samples called: 26444369, samples/s: 7601.92
Fini

### Eine erste Analyse mit Dotter
n dem Verzeichnis *basecalled/pass* findet sich eine Datei im FASTQ-Format, in der die DNA-Sequenzen der Reads enthalten sind. Eigentlich können wir erst jetzt wirklich von Reads sprechen. Diese Datei wandeln wir jetzt in eine Datei im FASTA-Format um. Dazu nutzen wir die Programmiersprache AWK, mit der einfach Textformatierungen durchgeführt werden können. Hier weisen wir awk an ab Zeile 1 bzw. Zeile 2 jede vierte Zeilen auszugeben. Mit `head -10` begrenzen wir die Ausgabe auf zehn Zeilen. 

In [18]:
awk 'NR%4==1 {print ">"$1} NR%4==2 {print $0}' BasecalledAll/pass/fastq_runid_c9ffe0f31e537612d2cbf4c0f7e050d9b0d928a0_0_0.fastq | head -10 > rohreads10.fasta

Dotter ist ein grafisches Dotplot-Programm für den detaillierten Vergleich von zwei DNA-Sequenzen. Dabei wird jedes Nukleotid in einer Sequenz mit jedem Nukleotid in der anderen Sequenz verglichen. Die erste Sequenz verläuft entlang der X-Achse und die zweite Sequenz (oder mehr) entlang der Y-Achse. In Bereichen, in denen sich die beiden Sequenzen ähneln, verläuft eine Diagonale durch die Matrix. Wenn Sie eine Sequenz mit sich selbst vergleichen würden, erhielten Sie eine Winkelhalbierende. Der Programmaufruf wird von den FASTA-Dateien der Referenz- und den zu vergleichenden Sequenzen gefolgt.

Dotter starten Sie mit `dotter pBR322.fasta rohreads10.fasta` in der Kommandozeile. Der Terminal ist jetzt nicht mehr eingabebereit – das Eingabeprompt ($) erscheint erst wieder nach dem Schließen von Dotter. Wir erhalten in einem graphischen Fenster einen Dotplot, der uns einen ersten Eindruck zu unserem Ergebnis liefert.

![dotter](dotter.png))

Auf der X-Achse ist die Sequenz des Plasmids, auf den Y-Achsen jeweils durch grüne Linien getrennt die Sequenz der Reads aufgetragen. Stimmen Nukleotide auf der X- und Y-Achse überein, dann wird ein Punkt gezeichnet; daher der Name Dotplot. 
.

### Demultiplexen mit PoreChop
Wenn verschiedene Experimente durchgeführt, jeweils durch Barcodes voneinander getrennt und auf einer Flowcell sequenziert wurden, landen alle Reads in einer FAST5- bzw. FASTQ-Datei. Diese können anschließend mit dem Programm` porecho`p wieder voneinander getrennt werde:es:

In [1]:
porechop --input BasecalledAll/pass/fastq_runid_c9ffe0f31e537612d2cbf4c0f7e050d9b0d928a0_0_0.fastq --barcode_dir BarCodesAll


Loading reads
BasecalledAll/pass/fastq_runid_c9ffe0f31e537612d2cbf4c0f7e050d9b0d928a0_0_0.fastq
666 reads loaded


Looking for known adapter sets
666 / 666 (100.0%)

Barcodes determined to be in forward orientation
                                        Best               
                                        read       Best    
                                        start      read end
  Set                                   %ID        %ID     
  SQK-NSK007                                79.3       75.0
  Rapid                                    100.0        0.0
  RBK004_upstream                           86.8        0.0
  SQK-MAP006                                71.0       72.0
  SQK-MAP006 short                          80.0       72.0
  PCR adapters 1                            73.9       77.3
  PCR adapters 2                            75.0       72.0
  PCR adapters 3                            75.0       73.9
  1D^2 part 1                               69.7       69.0
  1D

Die Option `--barcode_dir` bezeichnet das Verzeichnis, in welchem die gefundenen Barcodes gespeichert werden. Die gesamte Ausgabe ist recht lang. Entscheidend sind die letzten Zeilen die zeigen, wieviel Reads jeweils Barcodes zugeordnet werden konnten.

### Qualitätskontrolle und Filtern der Reads
Die Qualität der vom MinION erzeugten Reads wird variieren. Wenn die Qualität zu schlecht ist, kann eine Übereinstimmung mit der Plasmidsequenz später nicht detektiert werden. Die Qualität wird mit dem sogenannten PHRED-Quality-Score (oder einfach Quality-Score) angegeben. Er steht in einem logarithmischen Verhältnis zu den Fehlerwahrscheinlichkeiten der Basenaufrufe beim Base-Calling. Ein Quality-Score von 10 besagt, dass 90% der Basen korrekt sind; bei einem Wert von 20 sind es 99%.
Zusätzlich zur Qualität filtern wir auch nach der Readlänge. Bei der Erstellung der Sequenzierbibliothek können sich DNA-Moleküle aneinanderlagern: Es entstehen sogenannte Konkatemere.
 
Zunächst downloaden wir das Programm qFilter und machen es ausführbar:

In [2]:
wget "https://github.com/awkologist/qfilter/raw/main/qfilter"

--2024-01-12 13:14:29--  https://github.com/awkologist/qfilter/raw/main/qfilter
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/qfilter/main/qfilter [following]
--2024-01-12 13:14:29--  https://raw.githubusercontent.com/awkologist/qfilter/main/qfilter
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5807 (5.7K) [text/plain]
Saving to: ‘qfilter’

qfilter             100%[===================>]   5.67K  --.-KB/s    in 0.002s  

2024-01-12 13:14:29 (2.25 MB/s) - ‘qfilter’ saved [5807/5807]



In [3]:
chmod u+x qfilter

Anschließend filtern wir die Reads. Hierbei nutzen wir eine For-Schleife, mit der alle drei Dateien *BC01.fastq*, *BC02.fastq* und *BC03.fastq* nacheinander abgearbeitet werden. Die For-Schleife hat das Format: `for` Variable `in` Liste`; do` Befehl(e)`; done` Als Variable verwenden wir *i*, die nacheinander die Werte Eins bis Drei annimmt. Mit *$i* wird der Variablenwert wieder aufgerufen.

Mit `-m` geben wir den minimalen durchschnittliches PHRED Score eines Reads an, mit `-x` die min. Readlänge und mit `-y` die maximale Readlänge.

In [8]:
for i in 1 2 3; do ./qfilter -m 10 -x 1000 -y 4361 BarCodesAll/BC0$i.fastq > bc0$i.all.fastq ;done

In [9]:
wc -l bc0[123].all.fastq

    156 bc01.all.fastq
     68 bc02.all.fastq
   1604 bc03.all.fastq
   1828 total


### Mappen der Reads auf das Plasmid
Für das Mappen der Reads auf das Plasmid pBR322 verwenden wir das Programm Minimap. Mit "mapping" ist der Prozess gemeint, der guckt an welchen Positionen in der Plasmidsequenz die vom MinION erzeugten Reads passen. Es wird also mit jedem Read ein Sequenz-Alignment durchgeführt. Auch hier verwenden wir wieder eine For-Schleife, in diesem Fall mit Dateinamen als Liste; das Fragezeichen steht dabei als Joker für ein beliebiges Zeichen.

In [10]:
for i in bc0?.all.fastq; do minimap2 -a pBR322.fasta $i > $i.mapped.sam; done

[M::mm_idx_gen::0.007*0.46] collected minimizers
[M::mm_idx_gen::0.008*0.61] sorted minimizers
[M::main::0.008*0.62] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.008*0.62] mid_occ = 10
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.008*0.62] distinct minimizers: 816 (100.00% are singletons); average occurrences: 1.000; average spacing: 5.344; total length: 4361
[M::worker_pipeline::0.025*1.45] mapped 39 sequences
[M::main] CMD: minimap2 -a pBR322.fasta bc01.all.fastq
[M::main] Real time: 0.026 sec; CPU: 0.037 sec; Peak RSS: 0.005 GB
...


Die Option `-a` weist Minimap2 an, das Ergebnis im sogenannten SAM-Format auszugeben. Normalerweise würde die Ausgabe auf den Bildschirm erfolgen. Ich habe die Ausgabe wieder beschnitten. Mit der Umleitung `>` (redirection) leiten wir die Daten in eine Datei, die wir *mapped.sam* benennen.

### Vorbereitung für die Visualisierung mit IGV
Der Integrated Genome Viewer (IGV) ist ein Programm für die Betrachtung der gemappten Reads. Dazu wird zunächst die Referenz geladen (Genome) und dann die Reads (Track). Das Genom ist in unserem Fall das Plasmid pBR322 in der Datei *pBR322.fasta* und die gemappten Reads befinden sich in der zuvor mittelsMminimap2 erstellten Datei *mapped.sam*. Diese muss für IGV noch sortiert, indexiert und in ein binäres Format gewandelt werden. Das machen wir mit dem Programm `samtools`. Wieder nutzen wir die For-Schleife, um alle drei Dateien jeweils zu sortieren, in das binäre BAM-Format umzuwandeln und anschließend zu indexieren.

In [11]:
for i in bc0?.all.fastq.mapped.sam; do samtools sort $i -o $i.sorted.bam; samtools index $i.sorted.bam; done

Das Ergebnis ist die sortierten Dateien *bc0[123].all.fastq.mapped.sorted.bam* mit ihrem Index in der Datei *bc0[123].all.fastq.mapped.sorted.bam.bai*, die IGV im Hintergrund laden wird. Mit "[123]" will ich andeuten, dass es sich um jeweils drei Dateien handelt.
Das Plasmid (Genome) indexiert IGV automatisch beim Laden :-)

### Visualisierung mit IGV
Der Integrated Genome Viewer(`igv`) ist ein vielseitiges JAVA-Programm zur Darstellung genomischer Daten 
Nach dem Starten des Programms ist der Terminal wie bei Dotter belegt und kann erst wieder mit Befehlen gefüttert werden, wenn das IGV-Fenster geschlossen wurde.

Im IGV-Fenster geben wir zunächst das Referenzgenom im Menüpunkt **Genomes > Load Genomes from File ...** an. In unserem bescheidenen Fall handelt es sich um das Plasmid pBR322, dessen Sequenz in der FASTA-Datei *pBR322.fasta* gespeichert ist. Danach laden wir die Reads der drei Experimente im BAM-Format (*bc0[123].all.fastq.mapped.ont.sam.sorted.bam*) im Menüpunlt **File > Load from File ...** Mit jedem Laden eines Datensatzes füllt sich das Fenster. Für jede BAM-Datei (respektive Barcode bzw. Experiment) sehen wir die Reads und darüber die Trefferdichte (coverage), die diese Reads erzeugen. Wundern Sie sich nicht, wenn es bei Ihnen etwas anders aussieht.

![igv](igv.png)

Es fällt auf, dass wir im oberen Datensatz eine Lücke bei 670 Basenpaaren sehen. Im mittleren Datensatz kommt eine Lücke bei etwa 3.760 Basenpaaren hinzu. Im unteren Datensatz dagegen ist die gesamte Plasmidsequenz von Reads abgedeckt. 
Was sagt uns das? Im

- oberen Datensatz wurde das Plasmid einmal
- im mittleren Datensatz zweimal (Cas9 und PstI)
- und im unteren Datensatz gar nicht geschnitten (unverdaute Kontrolle)

Bedenken Sie, dass keine Reads über Schnittstellen liegen können. Im oberen Datensatz finden wir eine Ausnahme: Ein Read überspannt die Schnittstelle bei 670 bp, was möglicherweise für einen teilweise unvollständigen Verdau spricht
In der dargestellten Analyse haben wir nur die ersten tausend Reads verwendet, die nach rund zehn Minuten Sequenzierung vom MinION auf dem angeschlossenen Computer gespeichert wurden. Das ist ein Bruchteil aller Daten.
